#### Insert Library dan Path

In [8]:
import os
import csv

label_folder = r"C:\Users\KRTI\Downloads\aas vision\archive\Indonesian License Plate Recognition Dataset\labels\test"
image_folder = r"C:\Users\KRTI\Downloads\aas vision\archive\Indonesian License Plate Recognition Dataset\images\test"
class_file = r"C:\Users\KRTI\Downloads\aas vision\archive\Indonesian License Plate Recognition Dataset\classes.names"
output_csv = "ground_truth.csv"

#### Load Mapping class_id → Karakter

In [9]:
with open(class_file, "r", encoding="utf-8") as f:
    classes = [line.strip() for line in f.readlines()]

print("Total kelas:", len(classes))
print("Contoh kelas:", classes[:10])

Total kelas: 36
Contoh kelas: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


#### Fungsi Parse Label File (YOLO → Teks)

In [10]:
def parse_label_file(file_path):
    chars = []
    with open(file_path, "r") as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) != 5:
                continue
            class_id, x_center = int(parts[0]), float(parts[1])
            char = classes[class_id]
            chars.append((x_center, char))
    chars.sort(key=lambda x: x[0])
    text = "".join([c[1] for c in chars])
    return text

#### Proses Semua File Label dan Cocokkan dengan Gambar

In [11]:
rows = []

for file_name in os.listdir(label_folder):
    if not file_name.endswith(".txt"):
        continue
    label_path = os.path.join(label_folder, file_name)
    image_name = file_name.replace(".txt", ".jpg")
    image_path = os.path.join(image_folder, image_name)
    
    if not os.path.exists(image_path):
        print(f"[!] Gambar tidak ditemukan untuk {file_name}, dilewati.")
        continue
    
    gt_text = parse_label_file(label_path)
    rows.append([image_name, gt_text])

#### Simpan ke CSV

In [12]:
with open(output_csv, "w", newline='', encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["image", "ground_truth"])
    writer.writerows(rows)

print(f"[✓] ground_truth.csv berhasil dibuat: {len(rows)} entri.")

[✓] ground_truth.csv berhasil dibuat: 197 entri.


#### Import Library

In [13]:
import pandas as pd
import lmstudio as lms
from evaluate import load

#### Konfigurasi dan Prompt

In [14]:
ground_truth_file = "ground_truth.csv"
output_csv = "ocr_results.csv"

prompt = "What is the license plate number shown in this image? Respond only with the license plate characters, without any spaces, or punctuation. Do not include the expiration date."

#### Load Ground Truth

In [15]:
df_gt = pd.read_csv(ground_truth_file)
ground_truth_dict = dict(zip(df_gt["image"], df_gt["ground_truth"]))

#### Inisialisasi Model

In [ ]:
model = lms.llm("qwen2-vl-2b-instruct")

#### Proses OCR untuk Setiap Gambar

In [ ]:
results = []
cer_scores = [] 

for filename, true_text in ground_truth_dict.items():
    image_path = os.path.join(image_folder, filename)
    if not os.path.exists(image_path):
        print(f"[!] File tidak ditemukan: {image_path}")
        continue

    try:
        image_handle = lms.prepare_image(image_path)
        chat = lms.Chat()
        chat.add_user_message(prompt, images=[image_handle])
        response = model.respond(chat)

        pred_text = response.content.strip().replace(" ", "").upper()
        true_text = true_text.strip().replace(" ", "").upper()

        # Hitung CER per baris
        cer_metric = load("cer")
        cer = cer_metric.compute(predictions=[pred_text], references=[true_text])

        print(f"[✓] {filename}: GT={true_text}, Pred={pred_text}, CER={cer:.4f}")
        cer_scores.append(cer)

        results.append({
            "image": filename,
            "ground_truth": true_text,
            "prediction": pred_text,
            "CER_score": round(cer, 4)
        })

    except Exception as e:
        print(f"[X] Error saat memproses {filename}: {e}")
        cer_scores.append(1.0)
        results.append({
            "image": filename,
            "ground_truth": true_text,
            "prediction": "ERROR",
            "CER_score": 1.0
        })

[✓] test001_1.jpg: GT=B9140BCD, Pred=B9140BCD, CER=0.0000
[✓] test001_2.jpg: GT=B2407UZO, Pred=B2407UZ0, CER=0.1250
[✓] test001_3.jpg: GT=B2842PKM, Pred=B2842PKM, CER=0.0000
[✓] test002_1.jpg: GT=BG1352AE, Pred=BG1352AE, CER=0.0000
[✓] test003_1.jpg: GT=B2634UZF, Pred=B2634UZF, CER=0.0000
[✓] test003_2.jpg: GT=B1995JVK, Pred=B1995JVK, CER=0.0000
[✓] test004_1.jpg: GT=B9062VEH, Pred=89062VEH, CER=0.1250
[✓] test005_1.jpg: GT=DD8798KM, Pred=DD2798KM, CER=0.1250
[✓] test006_1.jpg: GT=T1329KC, Pred=T1329KC, CER=0.0000
[✓] test007_1.jpg: GT=AD8865EE, Pred=AD8865EE, CER=0.0000
[✓] test008_1.jpg: GT=DK1157AAB, Pred=DK1157AAB, CER=0.0000
[✓] test008_2.jpg: GT=AA1997FE, Pred=AA1997FE, CER=0.0000
[✓] test009_1.jpg: GT=H8518NA, Pred=H8518NA, CER=0.0000
[✓] test009_2.jpg: GT=K1649GB, Pred=K1649GB, CER=0.0000
[✓] test010_1.jpg: GT=B9416PCN, Pred=B9416PCN, CER=0.0000
[✓] test010_2.jpg: GT=H1147UY, Pred=H1147UY, CER=0.0000
[✓] test011_1.jpg: GT=H1691IL, Pred=H1691IL, CER=0.0000
[✓] test011_2.jpg: GT=

#### Hitung CER Rata-rata

In [ ]:
average_cer = sum(cer_scores) / len(cer_scores)
print(f"\n📊 Rata-rata CER (Average CER): {average_cer:.4f}")


📊 Rata-rata CER (Average CER): 0.0384


#### Simpan ke CSV

In [ ]:
df = pd.DataFrame(results)
df.to_csv(output_csv, index=False)
print(f"\n📁 Hasil disimpan ke: {output_csv}")


📁 Hasil disimpan ke: ocr_results.csv
